# Lab 3: Time Series Forecasting for BTC Hourly Data

This notebook performs the tasks outlined in Lab 3:
1.  **Choose Forecasting Methods**: Based on Lab 2 insights, we select SARIMA, Prophet, and ETS.
2.  **Check for Heteroscedasticity**: Analyze residuals for non-constant variance.
3.  **Build Models & Check Quality**: Fit models on a training set and evaluate in-sample performance.
4.  **Generate Forecasts & Check Quality**: Predict on a test set and evaluate out-of-sample performance.
5.  **Tabulate Statistical Characteristics**.
6.  **Display Intermediate Graphs**.